In [5]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import shap
import math
from itertools import combinations

# Dataset
X = pd.DataFrame({
    "Feature1": [1, 2, 3],
    "Feature2": [4, 5, 6],
    "Feature3": [7, 8, 9]
})
y = 1*X["Feature1"] + 2*X["Feature2"] + 3*X["Feature3"] + 5

# Train model
model = LinearRegression()
model.fit(X, y)
preds = model.predict(X)

# Power set function
def powerset(s):
    return [list(comb) for i in range(len(s)+1) for comb in combinations(s, i)]

# Manual SHAP with logging
def manual_shap_values_verbose(model, X_row, X_background):
    feature_names = list(X_row.index)
    baseline = X_background.mean().values.reshape(1, -1)
    n = len(feature_names)
    
    log_rows = []
    shap_values = dict.fromkeys(feature_names, 0.0)
    
    for feature in feature_names:
        contribution = 0
        subsets = powerset([f for f in feature_names if f != feature])
        
        for subset in subsets:
            with_feature = subset + [feature]
            without_feature = subset
            
            x_with = baseline.copy()
            x_without = baseline.copy()

            for f in with_feature:
                x_with[0, feature_names.index(f)] = X_row[f]
            for f in without_feature:
                x_without[0, feature_names.index(f)] = X_row[f]

            x_with_df = pd.DataFrame(x_with, columns=feature_names)
            x_without_df = pd.DataFrame(x_without, columns=feature_names)
            pred_with = model.predict(x_with_df)[0]
            pred_without = model.predict(x_without_df)[0]
            marginal_contrib = pred_with - pred_without

            weight = math.factorial(len(subset)) * math.factorial(n - len(subset) - 1) / math.factorial(n)
            weighted_contrib = weight * marginal_contrib
            contribution += weighted_contrib

            log_rows.append({
                "Feature": feature,
                "Subset": subset,
                "With Feature Pred": pred_with,
                "Without Feature Pred": pred_without,
                "Marginal Contribution": marginal_contrib,
                "Weight": weight,
                "Weighted Contribution": weighted_contrib
            })
        
        shap_values[feature] = contribution
    
    log_df = pd.DataFrame(log_rows)
    return shap_values, log_df

# Run for first row
manual_shap, log_table = manual_shap_values_verbose(model, X.iloc[0], X)

# SHAP package comparison
explainer = shap.Explainer(model, X)
shap_values_package = explainer(X)

# Final summary table
summary = pd.DataFrame({
    "Feature1": [X.iloc[0]["Feature1"]],
    "Feature2": [X.iloc[0]["Feature2"]],
    "Feature3": [X.iloc[0]["Feature3"]],
    "Prediction": [preds[0]],
    "Manual_SHAP_Feature1": [manual_shap["Feature1"]],
    "Manual_SHAP_Feature2": [manual_shap["Feature2"]],
    "Manual_SHAP_Feature3": [manual_shap["Feature3"]],
    "SHAP_Package_Feature1": [shap_values_package.values[0][0]],
    "SHAP_Package_Feature2": [shap_values_package.values[0][1]],
    "SHAP_Package_Feature3": [shap_values_package.values[0][2]],
})

# Show results
print("\n=== Final Summary ===")
print(summary)

print("\n=== SHAP Step-by-Step Breakdown ===")
print(log_table.to_string(index=False))



=== Final Summary ===
   Feature1  Feature2  Feature3  Prediction  Manual_SHAP_Feature1  \
0         1         4         7        35.0                  -2.0   

   Manual_SHAP_Feature2  Manual_SHAP_Feature3  SHAP_Package_Feature1  \
0                  -2.0                  -2.0                   -2.0   

   SHAP_Package_Feature2  SHAP_Package_Feature3  
0                   -2.0                   -2.0  

=== SHAP Step-by-Step Breakdown ===
 Feature               Subset  With Feature Pred  Without Feature Pred  Marginal Contribution   Weight  Weighted Contribution
Feature1                   []               39.0                  41.0                   -2.0 0.333333              -0.666667
Feature1           [Feature2]               37.0                  39.0                   -2.0 0.166667              -0.333333
Feature1           [Feature3]               37.0                  39.0                   -2.0 0.166667              -0.333333
Feature1 [Feature2, Feature3]               35.0    